In [2]:
import pandas as pd

In [5]:
hb_raw = pd.read_table('results/Hyperband_server.csv', sep=',')
hb_raw.head()

,learning_rate,bracket,n_i,r_i,train_loss,train_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
0,0.248571,4,81.0,1.0,1.688257,37.726562,1.377976,48.450000,1.378134,47.43
1,0.022676,4,81.0,1.0,2.084372,25.953125,1.747915,40.366667,1.743672,39.99
2,0.256718,4,81.0,1.0,1.716258,37.085938,1.433188,42.400000,1.467930,40.03
3,0.058977,4,81.0,1.0,1.898225,31.335938,1.492426,47.283333,1.516912,45.87
4,0.053501,4,81.0,1.0,1.917115,31.742188,1.508192,46.933333,1.509761,46.77


In [6]:
sh_raw = pd.read_table('results/SuccessiveHalving.csv', sep=',')
sh_raw.head()

,learning_rate,bracket,n_i,r_i,train_loss,train_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
0,0.248571,4,162,81,1.004724,62.871926,0.731205,72.650000,0.770260,71.34
1,0.022676,4,162,81,1.126467,58.743497,0.851161,67.866667,0.876698,66.88
2,0.256718,4,162,81,1.001346,62.888521,0.817981,70.116667,0.840518,69.04
3,0.058977,4,162,81,1.043664,61.516398,0.797966,71.516667,0.800360,71.46
4,0.053501,4,162,81,1.049321,61.222233,0.813515,69.766667,0.818799,69.72
